In [1]:
# Only install what might be missing on Colab
!pip install noisereduce --quiet


In [7]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import soundfile as sf
import noisereduce as nr
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, GRU, TimeDistributed
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, classification_report


In [8]:
# Colab should have a GPU by default if you select Runtime > Change runtime type > GPU
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.19.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
# Clone the Coswara dataset from GitHub
!git clone https://github.com/iiscleap/Coswara-Data.git /content/Coswara-Data


Cloning into '/content/Coswara-Data'...
remote: Enumerating objects: 995, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 995 (delta 58), reused 45 (delta 45), pack-reused 928 (from 3)
Receiving objects: 100% (995/995), 15.97 GiB | 42.82 MiB/s, done.
Resolving deltas: 100% (422/422), done.
Updating files: 100% (336/336), done.


In [10]:
import pandas as pd

# Load combined metadata CSV
metadata_path = '/content/Coswara-Data/combined_data.csv'
metadata = pd.read_csv(metadata_path)

# Quick look at first rows
metadata.head()


,id,a,covid_status,record_date,ep,g,l_c,l_l,l_s,rU,...,vacc,bd,others_resp,ftg,st,ihd,asthma,others_preexist,cld,pneumonia
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,28,healthy,2020-04-23,y,male,India,Anantapur,Andhra Pradesh,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,25,healthy,2020-04-20,y,male,India,BENGALURU URBAN,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,28,healthy,2020-04-24,y,female,United States,Pittsburgh,Pennsylvania,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,26,healthy,2020-04-23,y,male,India,Bangalore,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,32,healthy,2020-04-22,y,male,India,Nalanda,Bihar,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
%cd /content/Coswara-Data
!python extract_data.py


Streaming output truncated to the last 5000 lines.
20220224/IBZq01pmKeQaDwQrGq5oZ6ZbYuH3/metadata.json
20220224/IBZq01pmKeQaDwQrGq5oZ6ZbYuH3/vowel-e.wav
20220224/IBZq01pmKeQaDwQrGq5oZ6ZbYuH3/breathing-shallow.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/vowel-a.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/counting-normal.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/cough-heavy.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/counting-fast.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/cough-shallow.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/vowel-o.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/breathing-deep.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/metadata.json
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/vowel-e.wav
20220224/xSf245CWofXICvNTo7WVBwfaBmf1/breathing-shallow.wav
20220224/xxbH7S4Z12grhWYc3aEiMzoVzbY2/
20220224/xxbH7S4Z12grhWYc3aEiMzoVzbY2/vowel-a.wav
20220224/xxbH7S4Z12grhWYc3aEiMzoVzbY2/counting-normal.wav
20220224/xxbH7S4Z12grhWYc3aEiMzoVzbY2/cough-heavy.wav
2022

In [13]:
!mv /content/Coswara-Data/Extracted_data /content/


In [14]:
!rm -rf /content/Coswara-Data


In [15]:
%cd /content/Coswara-Data


[Errno 2] No such file or directory: '/content/Coswara-Data'
/content/Coswara-Data


In [17]:
DATA_PATH = '/content/Extracted_data'


In [20]:
!wget https://github.com/iiscleap/Coswara-Data/raw/main/combined_data.csv -O combined_data.csv


--2025-10-13 15:09:45--  https://github.com/iiscleap/Coswara-Data/raw/main/combined_data.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-10-13 15:09:45 ERROR 404: Not Found.



In [29]:
from google.colab import files
uploaded = files.upload()  # then select the CSV


Saving combined_data.csv to combined_data (1).csv


In [33]:
import os
print("Current directory:", os.getcwd())
!ls -lh


Current directory: /content
total 360K
-rw-r--r--  1 root root 351K Oct 13 15:14 combined_data.csv
drwxr-xr-x 45 root root 4.0K Oct 13 14:54 Extracted_data
drwxr-xr-x  1 root root 4.0K Oct  9 13:36 sample_data


In [69]:
import pandas as pd

metadata = pd.read_csv("combined_data.csv")

# Filter for healthy or COVID-positive participants
covid_positive_status = ['positive_moderate', 'positive_mild', 'positive_asymp']
healthy_status = ['healthy']

metadata_filtered = metadata[metadata['covid_status'].isin(covid_positive_status + healthy_status)].copy()

# Create binary labels
metadata_filtered['label'] = metadata_filtered['covid_status'].apply(lambda x: 1 if x in covid_positive_status else 0)

# Check distribution
print(metadata_filtered['label'].value_counts())


label
0    1433
1     681
Name: count, dtype: int64


In [70]:
import os

sample_path = '/content/Extracted_data/20200413/0Rlzhiz6bybk77wdLjxwy7yLDhg1'
print("Files in participant folder:")
print(os.listdir(sample_path))


Files in participant folder:
['breathing-deep.wav', 'metadata.json', 'vowel-e.wav', 'cough-shallow.wav', 'cough-heavy.wav', 'counting-fast.wav', 'breathing-shallow.wav', 'vowel-a.wav', 'vowel-o.wav', 'counting-normal.wav']


In [71]:
import json
import os

# Example participant folder
participant_folder = "/content/Extracted_data/20200413/0Rlzhiz6bybk77wdLjxwy7yLDhg1"
metadata_file = os.path.join(participant_folder, "metadata.json")

# Read JSON
with open(metadata_file, "r") as f:
    participant_metadata = json.load(f)

# Inspect content
print(json.dumps(participant_metadata, indent=4))


{
    "id": "0Rlzhiz6bybk77wdLjxwy7yLDhg1",
    "a": 37,
    "covid_status": "no_resp_illness_exposed",
    "iF": true,
    "ep": "y",
    "g": "male",
    "l_s": "Maharashtra",
    "dT": "web",
    "l_c": "India"
}


In [72]:
import pandas as pd

metadata = pd.read_csv("combined_data.csv")


In [73]:
# 0 = healthy, 1 = any non-healthy
metadata['label'] = metadata['covid_status'].apply(lambda x: 0 if x.lower() == 'healthy' else 1)

# Check distribution
print(metadata['label'].value_counts())
print(metadata['covid_status'].value_counts())


label
0    1433
1    1313
Name: count, dtype: int64
covid_status
healthy                        1433
positive_mild                   426
no_resp_illness_exposed         248
positive_moderate               165
resp_illness_not_identified     157
recovered_full                  146
positive_asymp                   90
under_validation                 81
Name: count, dtype: int64


In [74]:
metadata.shape


(2746, 37)

In [75]:
metadata['covid_status'].value_counts()


,count
covid_status,
healthy,1433
positive_mild,426
no_resp_illness_exposed,248
positive_moderate,165
resp_illness_not_identified,157
recovered_full,146
positive_asymp,90
under_validation,81


In [76]:
import os

def get_cough_paths(row):
    folder = os.path.join("/content/Extracted_data", str(row['record_date']).replace('-', ''), row['id'])
    files = ["cough-shallow.wav", "cough-heavy.wav"]
    return [os.path.join(folder, f) for f in files if os.path.exists(os.path.join(folder, f))]

metadata['cough_files'] = metadata.apply(get_cough_paths, axis=1)


In [77]:
metadata.shape

(2746, 38)

In [80]:
import os
import glob
import pandas as pd

data_root = "/content/Extracted_data"
metadata = pd.read_csv("combined_data.csv")

# Make a list of cough recordings for each participant
cough_files_list = []

for pid in metadata['id']:
    participant_folder = os.path.join(data_root, "*", pid)  # wildcard for date folders
    folders = glob.glob(participant_folder)

    cough_paths = []
    for folder in folders:
        for f in ["cough-shallow.wav", "cough-heavy.wav"]:
            path = os.path.join(folder, f)
            if os.path.exists(path):
                cough_paths.append(path)

    cough_files_list.append(cough_paths)

metadata['cough_files'] = cough_files_list

# Filter participants with at least one cough recording
metadata = metadata[metadata['cough_files'].map(len) > 0]


In [81]:
metadata.shape


(2746, 37)

In [79]:
metadata['covid_status'].value_counts()


,count
covid_status,
healthy,1433
positive_mild,426
no_resp_illness_exposed,248
positive_moderate,165
resp_illness_not_identified,157
recovered_full,146
positive_asymp,90
under_validation,81


In [83]:
# Number of participants with at least one cough recording
num_with_cough = (metadata['cough_files'].map(len) > 0).sum()
print(f"Participants with cough recordings: {num_with_cough}")

# Total participants
total = len(metadata)
print(f"Total participants: {total}")

# Fraction
print(f"Fraction with cough recordings: {num_with_cough/total:.2%}")
# Count number of cough files per participant
metadata['num_cough_files'] = metadata['cough_files'].map(len)

# Distribution of number of cough files
cough_count_distribution = metadata['num_cough_files'].value_counts().sort_index()
print(cough_count_distribution)



Participants with cough recordings: 2746
Total participants: 2746
Fraction with cough recordings: 100.00%
num_cough_files
1       1
2    2745
Name: count, dtype: int64


In [84]:
metadata = metadata[metadata['num_cough_files'] > 1]


In [85]:
metadata.shape

(2745, 38)

In [88]:
import glob
import os

def get_cough_files(participant_id):
    # This finds the exact folder for the participant
    folders = glob.glob(f"/content/Extracted_data/*/{participant_id}")
    files = []
    if folders:  # folder exists
        folder = folders[0]  # take the first match
        for f in os.listdir(folder):
            if "cough" in f:
                files.append(os.path.join(folder, f))
    return files

metadata['cough_files'] = metadata['id'].apply(get_cough_files)


In [89]:
metadata.shape

(2745, 38)

In [92]:
# List all column names
print(metadata.columns)


Index(['id', 'a', 'covid_status', 'record_date', 'ep', 'g', 'l_c', 'l_l',
       'l_s', 'rU', 'smoker', 'cold', 'ht', 'diabetes', 'cough', 'ctDate',
       'ctScan', 'ctScore', 'diarrhoea', 'fever', 'loss_of_smell', 'mp',
       'testType', 'test_date', 'test_status', 'um', 'vacc', 'bd',
       'others_resp', 'ftg', 'st', 'ihd', 'asthma', 'others_preexist', 'cld',
       'pneumonia', 'cough_files', 'num_cough_files'],
      dtype='object')


In [93]:
# Map covid_status to binary labels
metadata['label'] = metadata['covid_status'].apply(lambda x: 0 if x == 'healthy' else 1)

# Quick check
metadata[['covid_status', 'label']].head()


,covid_status,label
0,healthy,0
1,healthy,0
2,healthy,0
3,healthy,0
4,healthy,0


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['label']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'label'}, axis=1)
              .sort_values('label', ascending=True))
  xs = counted['label']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('label', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('label')
_ = plt.ylabel('count()')

In [94]:
import librosa
import numpy as np
from tqdm import tqdm

all_features = []
all_labels = []

def extract_features(audio_paths, label):
    features = []
    labels = []
    for file_path in audio_paths:
        try:
            signal, sr = librosa.load(file_path, sr=22050)
            mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=128)
            log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
            features.append(log_mel_spec)
            labels.append(label)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    return features, labels

for idx, row in tqdm(metadata.iterrows(), total=metadata.shape[0], desc="Extracting features"):
    files = row['cough_files']
    lbl = row['label']  # 0 or 1
    feats, labs = extract_features(files, lbl)
    all_features.extend(feats)
    all_labels.extend(labs)

print(f"Total spectrograms extracted: {len(all_features)}")


Extracting features:   1%|▏         | 37/2745 [00:01<01:48, 25.03it/s]/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
Extracting features:  21%|██▏       | 586/2745 [00:24<01:04, 33.62it/s]/tmp/ipython-input-277499336.py:13: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=22050)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Extracting features:  22%|██▏       | 594/2745 [00:24<01:24, 25.48it/s]

Error processing /content/Extracted_data/20200820/5q8gLM0yCrgGCT8F9fWlH4ycl1D3/._cough-heavy.wav: 


Extracting features:  23%|██▎       | 630/2745 [00:26<01:15, 28.01it/s]/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
Extracting features:  92%|█████████▏| 2514/2745 [01:50<00:09, 24.52it/s]/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1882
  warnings.warn(
Extracting features: 100%|██████████| 2745/2745 [02:00<00:00, 22.82it/s]

Total spectrograms extracted: 5490


In [95]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# ------------------------------
# 1. Standardize spectrogram widths
# ------------------------------
# Find a suitable width (95th percentile)
spec_widths = [spec.shape[1] for spec in all_features]
TARGET_LEN = int(np.percentile(spec_widths, 95))
print(f"Target spectrogram width: {TARGET_LEN}")

def standardize_spectrogram(spec, target_len):
    """Pad or truncate a spectrogram to a fixed width."""
    current_len = spec.shape[1]
    if current_len > target_len:
        return spec[:, :target_len]
    elif current_len < target_len:
        pad_width = target_len - current_len
        return np.pad(spec, ((0,0),(0,pad_width)), mode='constant')
    else:
        return spec

# Apply standardization
X = np.array([standardize_spectrogram(s, TARGET_LEN) for s in all_features])
y = np.array(all_labels)

# Add channel dimension for CNN
X = X[..., np.newaxis]
print(f"Feature matrix shape: {X.shape}")
print(f"Label vector shape: {y.shape}")

# ------------------------------
# 2. Train/Test split with stratification
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

# ------------------------------
# 3. Normalize spectrograms to [0,1]
# ------------------------------
scaler = MinMaxScaler()

# Reshape to 2D for scaler: (samples, height*width)
train_shape = X_train.shape
test_shape = X_test.shape

X_train_reshaped = X_train.reshape(train_shape[0], -1)
X_test_reshaped = X_test.reshape(test_shape[0], -1)

# Fit scaler on training data only
scaler.fit(X_train_reshaped)

# Transform both train and test
X_train_scaled = scaler.transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

# Reshape back to original 4D format
X_train = X_train_scaled.reshape(train_shape)
X_test = X_test_scaled.reshape(test_shape)

print(f"Data ready for CNN:")
print(f"X_train: {X_train.shape}, min={X_train.min():.2f}, max={X_train.max():.2f}")
print(f"X_test: {X_test.shape}, min={X_test.min():.2f}, max={X_test.max():.2f}")


Target spectrogram width: 416
Feature matrix shape: (5490, 128, 416, 1)
Label vector shape: (5490,)
Training samples: 4392, Testing samples: 1098
Data ready for CNN:
X_train: (4392, 128, 416, 1), min=0.00, max=1.00
X_test: (1098, 128, 416, 1), min=0.00, max=1.00


In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# ------------------------------
# 1. Define the CNN
# ------------------------------
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 416, 1)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# ------------------------------
# 2. Compile the model
# ------------------------------
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# ------------------------------
# 3. Train the model
# ------------------------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 23s 97ms/step - accuracy: 0.5291 - loss: 0.7016 - val_accuracy: 0.5628 - val_loss: 0.6826
Epoch 2/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - accuracy: 0.5722 - loss: 0.6775 - val_accuracy: 0.5838 - val_loss: 0.6677
Epoch 3/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.5969 - loss: 0.6615 - val_accuracy: 0.6202 - val_loss: 0.6552
Epoch 4/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.6294 - loss: 0.6494 - val_accuracy: 0.6293 - val_loss: 0.6471
Epoch 5/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.6424 - loss: 0.6339 - val_accuracy: 0.6375 - val_loss: 0.6352
Epoch 6/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.6515 - loss: 0.6244 - val_accuracy: 0.6566 - val_loss: 0.6334
Epoch 7/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.6537 - loss: 0.6245 - val_accuracy: 0.6648 - val_loss: 0.6256
Epoch 8/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - accuracy: 0.6869 - loss: 0.5984 - val_ac

In [97]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,        # you can increase/decrease depending on performance
    batch_size=32     # adjust if GPU memory is tight
)


Epoch 1/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - accuracy: 0.8354 - loss: 0.3560 - val_accuracy: 0.6539 - val_loss: 0.7232
Epoch 2/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - accuracy: 0.8479 - loss: 0.3315 - val_accuracy: 0.6485 - val_loss: 0.7433
Epoch 3/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - accuracy: 0.8569 - loss: 0.3254 - val_accuracy: 0.6357 - val_loss: 0.7644
Epoch 4/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - accuracy: 0.8670 - loss: 0.2990 - val_accuracy: 0.6384 - val_loss: 0.7884
Epoch 5/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - accuracy: 0.8847 - loss: 0.2666 - val_accuracy: 0.6412 - val_loss: 0.8036
Epoch 6/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - accuracy: 0.8950 - loss: 0.2498 - val_accuracy: 0.6494 - val_loss: 0.8338
Epoch 7/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - accuracy: 0.8962 - loss: 0.2416 - val_accuracy: 0.6539 - val_loss: 0.8493
Epoch 8/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - accuracy: 0.9086 - loss: 0.2220 - val_accu

In [98]:
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.6469 - loss: 1.2390
Test Accuracy: 64.94%
Test Loss: 1.2024


In [99]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Get predicted probabilities
y_pred_probs = model.predict(X_test)

# Convert probabilities to class labels (0 or 1)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
Confusion Matrix:
[[376 197]
 [188 337]]

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.66      0.66       573
           1       0.63      0.64      0.64       525

    accuracy                           0.65      1098
   macro avg       0.65      0.65      0.65      1098
weighted avg       0.65      0.65      0.65      1098

